In [ ]:
from parameters import QudiHiraParameters
from src.data_handler import DataHandler
from src.analysis_logic import AnalysisLogic
import matplotlib.pyplot as plt

In [ ]:
# Create instance of DataHandler and give it the measurement folder
# The measurement folder must be located under the data folder specified in parameters.py (eg. Z:/Data/20220621_FR0612-F2-2S6_uhv)
data_handler = DataHandler(measurement_folder="20220621_FR0612-F2-2S6_uhv", params=ParametersQudiHira)

In [ ]:
# Automatically walk through measurement folders and extract a list
# of confocal images each being an instance of MeasurementDataclass
confocal_list = data_handler.load_measurements_into_dataclass_list(measurement_str="Confocal")

# Set up matplotlib figure
fig, ax = plt.subplots(nrows=len(confocal_list))

# Loop over all confocal images
for idx, confocal in enumerate(confocal_list):
    # Plot each confocal image on a subplot row
    ax[idx].imshow(confocal.data)
    # Extract the z-height param from the name of file
    z_height = confocal.get_param_from_filename(unit="um")
    ax[idx].set_title(f"Z-Height = {z_height}")

# Save output image
data_handler.save_figures(fig, filename="compare_confocals_at different_z_heights")

In [ ]:
rabi_list = data_handler.load_measurements_into_dataclass_list(measurement_str="Rabi")
analysis = AnalysisLogic()

fig, ax = plt.subplots(nrows=len(rabi_list))

for idx, rabi in enumerate(rabi_list):
    x = rabi["t(ns)"]
    y = rabi["spin_state"]

    # Fit data to an exponentially decaying sinusoid
    fit_x, fit_y, model = analysis.perform_fit(x, y, fit_function="sineexponentialdecay")
    
    ax[idx].plot(x, y)
    ax[idx].plot(fit_x, fit_y)
    
    # Extract the power param from the name of file
    power = rabi.get_param_from_filename(unit="dBm")
    
    # Title plot with power and T1rho time
    t1rho = model.best_fit.params["decay"]
    ax[idx].set_title(f"Power = {power}, T1rho = {t1rho}")

# Save output image
data_handler.save_figures(fig, filename="compare_rabis_at different_powers")